# 01) Importar librerias correspondientes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from sklearn.model_selection import GridSearchCV

# 02) Importar dataset de properati

In [4]:
properati = pd.read_csv(r'C:/Users/Miguel\Desktop/Ciencia de Datos/Clase_04_Regresion\clase_04-master/clusterai_2019_clase04_regresion_dataset_properati_caba.csv', delimiter=',', parse_dates = True)

In [5]:
np.shape(properati)

(25319, 13)

In [6]:
properati.columns

Index(['id', 'created_on', 'operation', 'property_type', 'neighbourhood',
       'lat', 'lon', 'price', 'surface_total_in_m2', 'surface_covered_in_m2',
       'floor', 'rooms', 'expenses'],
      dtype='object')

In [7]:
properati.head()

,id,created_on,operation,property_type,neighbourhood,lat,lon,price,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses
0,1649a3c35359d681abae4a63d9a8545054d2096b,2017-09-22,sell,house,Palermo Viejo,-34.588953,-58.429882,2000000.0,312.0,NaN,NaN,NaN,NaN
1,1ab035ccbe77702987c3f09850d12c634e612aa2,2017-09-01,sell,PH,Parque Chacabuco,-34.635866,-58.431048,131500.0,80.0,50.0,NaN,3.0,NaN
2,90fc7c61b6fbc6ef99368a21cd9ac0d30d54c697,2017-09-01,sell,apartment,Barracas,-34.644924,-58.371919,140000.0,125.0,85.0,NaN,NaN,NaN
3,c195e8f3a596ab49b95d699c07e1a04be503ba68,2017-09-01,sell,apartment,Recoleta,-34.596511,-58.385747,109000.0,45.0,45.0,2.0,2.0,3350.0
4,f891875ff44e6f99e30e101b3901d1e7510ef41b,2017-09-01,sell,apartment,Parque Patricios,-34.639173,-58.417069,94000.0,38.0,38.0,NaN,NaN,NaN


# 03) Revisar con que columnas trabajar para luego quitarle los NaNs (queremos predecir precio)

In [8]:
col_NaN = properati.isnull().any()
print(col_NaN,'\r\n')
cant_NaN = properati.isnull().sum()
print(cant_NaN,'\r\n')


id                       False
created_on               False
operation                False
property_type            False
neighbourhood            False
lat                       True
lon                       True
price                     True
surface_total_in_m2       True
surface_covered_in_m2     True
floor                     True
rooms                     True
expenses                  True
dtype: bool 

id                           0
created_on                   0
operation                    0
property_type                0
neighbourhood                0
lat                       1048
lon                       1048
price                     1928
surface_total_in_m2       3323
surface_covered_in_m2     2529
floor                    22093
rooms                     6607
expenses                 20289
dtype: int64 



In [9]:
properati1 = properati.drop(['id','created_on','operation','lat','lon','floor','expenses'], axis=1)

In [10]:
properati2 = properati1.dropna(subset = ['price','surface_total_in_m2','surface_covered_in_m2','rooms'])

In [11]:
properati2.shape

(16296, 6)

In [12]:
properati2.head()

,property_type,neighbourhood,price,surface_total_in_m2,surface_covered_in_m2,rooms
1,PH,Parque Chacabuco,131500.0,80.0,50.0,3.0
3,apartment,Recoleta,109000.0,45.0,45.0,2.0
5,apartment,Flores,160000.0,76.0,76.0,3.0
6,apartment,Caballito,160000.0,76.0,76.0,3.0
7,apartment,Villa Crespo,240435.0,101.0,75.0,3.0


In [13]:
properati2['rooms'].unique()

array([ 3.,  2.,  5.,  1.,  4.,  6.,  7.,  9.,  8., 10., 11., 12., 20.,
       30., 13., 15., 32., 24., 17., 34., 21., 25.])

# 04) Quitar Outliers de las variables que consideren utilizando cuantiles

In [14]:
price_q97 = properati2.price.quantile(0.975)
print("el cuantil 0.97 de la feature 'price' es = " + str(price_q97))

el cuantil 0.97 de la feature 'price' es = 2086406.625


In [15]:
surface_total_q97 = properati2.surface_total_in_m2.quantile(0.975)
print("el cuantil 0.97 de la feature 'surface_total_in_m2' es = " + str(surface_total_q97))

el cuantil 0.97 de la feature 'surface_total_in_m2' es = 379.625


In [16]:
surface_covered_q97 = properati2.surface_covered_in_m2.quantile(0.975)
print("el cuantil 0.97 de la feature 'surface_covered_in_m2' es = " + str(surface_covered_q97))

el cuantil 0.97 de la feature 'surface_covered_in_m2' es = 317.0


In [17]:
rooms_q97 = properati2.rooms.quantile(0.975)
print("el cuantil 0.97 de la feature 'rooms' es = " + str(rooms_q97))

el cuantil 0.97 de la feature 'rooms' es = 7.0


In [18]:
properati2_filt = properati2.loc[(properati2.price < price_q97) & (properati2.surface_total_in_m2 < surface_total_q97) & (properati2.surface_covered_in_m2 < surface_covered_q97) & (properati2.rooms < rooms_q97)]

In [19]:
properati2_filt.shape

(15237, 6)

# 05) Realizar feature engineering si fuera necesario

In [20]:
# dummies para la feature "neighbourhood"
neighs_dummie = pd.get_dummies(properati2_filt.neighbourhood)
neighs_dummie.head(3)

,Abasto,Agronomía,Almagro,Balvanera,Barracas,Barrio Norte,Belgrano,Boca,Boedo,Caballito,...,Villa General Mitre,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# dummies para la feature "property_typee"
property_dummie = pd.get_dummies(properati2_filt.property_type)
property_dummie.head(3)

,PH,apartment,house,store
1,1,0,0,0
3,0,1,0,0
5,0,1,0,0


In [22]:
properati2_filt = properati2_filt.join([neighs_dummie, property_dummie])
properati2_filt.shape

(15237, 71)

In [23]:
properati2_filt.head()

,property_type,neighbourhood,price,surface_total_in_m2,surface_covered_in_m2,rooms,Abasto,Agronomía,Almagro,Balvanera,...,Villa Pueyrredón,Villa Real,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,PH,apartment,house,store
1,PH,Parque Chacabuco,131500.0,80.0,50.0,3.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,apartment,Recoleta,109000.0,45.0,45.0,2.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,apartment,Flores,160000.0,76.0,76.0,3.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,apartment,Caballito,160000.0,76.0,76.0,3.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,apartment,Villa Crespo,240435.0,101.0,75.0,3.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [24]:
#dropeo las features para las cuales generé dummies

properati2_filt = properati2_filt.drop(['property_type','neighbourhood'], axis=1)

In [25]:
properati2_filt.head()

,price,surface_total_in_m2,surface_covered_in_m2,rooms,Abasto,Agronomía,Almagro,Balvanera,Barracas,Barrio Norte,...,Villa Pueyrredón,Villa Real,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,PH,apartment,house,store
1,131500.0,80.0,50.0,3.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,109000.0,45.0,45.0,2.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,160000.0,76.0,76.0,3.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,160000.0,76.0,76.0,3.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,240435.0,101.0,75.0,3.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# 06) Separar Y (price) & X (decidir que features utilizar para trabajar)
Recuerden realizar auto scaling sobre las variables en cuestion

In [26]:
y = np.array(properati2_filt[['price']])

In [27]:
x = properati2_filt.drop(['price'], axis=1)

In [28]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)

In [29]:
xtrain.head()

,surface_total_in_m2,surface_covered_in_m2,rooms,Abasto,Agronomía,Almagro,Balvanera,Barracas,Barrio Norte,Belgrano,...,Villa Pueyrredón,Villa Real,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,PH,apartment,house,store
11733,44.0,40.0,2.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
13318,116.0,75.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11691,102.0,42.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15039,42.0,42.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8081,134.0,129.0,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
xtrain.shape

(13713, 68)

In [31]:
# Step 4: auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler().fit(xtrain)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [32]:
# auto scalo mis muestras de train utilizando el scaler fiteado con el xtrain
xtrain_scal = scaler.transform(xtrain)  

In [33]:
# auto scalo mis muestras de test utilizando el scaler fiteado con el xtrain
xtest_scal = scaler.transform(xtest)  

# 07) Implementar un modelo de regresion para predecir el precio. De ser posible comparar performance de distintos modelos.

# LR

In [34]:
# 1) creo un modelo generico de regresion lineal
lr = LinearRegression()

In [35]:
# 2) Ajusto el modelo de regresion lineal utilizando el set de train, tanto las features X como las samples Y

#n_jobs para definir la cantidad de nucleos que se utilizan ('None = uno por default')

lr.fit(xtrain_scal, ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
# 3) Obtengo las predicciones que realiza mi modelo con las muestras de test, sin mostrarle las labels (Ytest)
# las predicciones las guardo en el vector "ypred"
ypred = lr.predict(xtest_scal)

In [37]:
np.shape(ypred)

(1524, 1)

In [38]:
# calculo el error de mi modelo con las muestras de train = error de train
#el resultado en la misma escala que el valor de la feature

# calculo el RMSE (root mean square error) del test

np.sqrt(mean_squared_error(ytest, ypred))

214643.17028987955

In [39]:
# calculo el MSE (mean square error) del test
mean_squared_error(ytest, ypred)

46071690552.090225

In [40]:
from sklearn.metrics import mean_absolute_error

In [41]:
# calculo el MAE (mean absolute error) del test
mean_absolute_error(ytest, ypred)

105890.1248661894

In [42]:
#Lo mismo para train
ytrainpred = lr.predict(xtrain_scal)

In [45]:
# calculo el RMSE (root mean square error) del train
np.sqrt(mean_squared_error(ytrain, ytrainpred))

231662.29836373395

In [46]:
# calculo el MAE (mean absolute error) del train
mean_absolute_error(ytrain, ytrainpred)

110156.36545488519

# KNN

In [47]:
# creamos nuestro modelo de KNN que se llama neigh
neigh = KNeighborsRegressor(weights = "distance")

In [49]:
 # creo un array de 5 posiciones con los valores del hiperparametro "k" que quiero probar por gridsearch
parameters_k = np.arange(17,49,7)
parameters_k

array([17, 24, 31, 38, 45])

In [50]:
# creo un diccionario "tuned_params" con los parametros que quiero optimizar, e indico que valores de
# ese parametro quiero probar
parameters_knn = [{'n_neighbors': parameters_k}]

In [51]:
# indico la cantidad de folds que quiero utilizar en mi training para cross validation
n_folds = 5

In [52]:
regresor_neigh = GridSearchCV(neigh, parameters_knn, refit = True, cv=n_folds, verbose=True)

In [53]:
# entreno modelo regresor con mi set de entrenamiento (tanto Xtrain como Ytrain)
regresor_neigh.fit(xtrain_scal, ytrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='distance'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': array([17, 24, 31, 38, 45])}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [54]:
# observo cual fue el parametro que "gano" en el cross validation y grid search
regresor_neigh.best_params_

{'n_neighbors': 31}

In [55]:
# calcula el R2
regresor_neigh.best_score_

0.6717200594201005

In [56]:
ypred2 = regresor_neigh.predict(xtest_scal)
ypred2

array([[153000.        ],
       [ 90436.74046953],
       [181885.65383563],
       ...,
       [102850.46250519],
       [111000.        ],
       [249250.        ]])

In [57]:
# calculo el RMSE (root mean square error) del test
np.sqrt(mean_squared_error(ytest, ypred2))

153963.01372779722

In [58]:
# calculo el MAE (mean absolute error) del test
mean_absolute_error(ytest, ypred2)

63395.661455611145

In [59]:
# calculo el MSE (mean square error) del test
mean_squared_error(ytest, ypred2)

23704609596.145878

In [60]:
#Lo mismo para train
ytrainpred2 = regresor_neigh.predict(xtrain_scal)

In [61]:
# calculo el RMSE (root mean square error) del train
np.sqrt(mean_squared_error(ytrain, ytrainpred2))

69728.32848852838

In [62]:
# calculo el MAE (mean absolute error) del train
mean_absolute_error(ytrain, ytrainpred2)

10661.483846150633

# SVR

In [88]:
# creo un modelo de SVR
svreg = SVR()

In [89]:
# cantidad de folds para el cross validation
n_folds = 5

In [90]:
# Distintos hiper parametros a probar en grid search (pruebo gridsearch por kernels separados)
parameters_svr_lin = [{'kernel':['linear'], 'C': [75, 150]}]
parameters_svr_rbf = [{'kernel':['rbf'] , 'C': [75,150],'gamma': [0.005,0.01] }]

regresor_svr_lin = GridSearchCV(svreg, parameters_svr_lin, cv=n_folds, verbose=True)
regresor_svr_rbf = GridSearchCV(svreg, parameters_svr_rbf, cv=n_folds, verbose=True)

In [91]:
# LINEAL

In [92]:
regresor_svr_lin.fit(xtrain_scal,ytrain.ravel())

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [75, 150], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [93]:
regresor_svr_lin.best_params_

{'C': 150, 'kernel': 'linear'}

In [94]:
regresor_svr_lin.best_score_

0.31890014127563193

In [95]:
ypred3 = regresor_svr_lin.predict(xtest_scal)

In [96]:
# calculo el RMSE (root mean square error) del test
np.sqrt(mean_squared_error(ytest, ypred3))

232062.3818038459

In [97]:
# calculo el MAE (mean absolute error) del test
mean_absolute_error(ytest, ypred3)

86058.93411523191

In [98]:
# calculo el MSE (mean square error) del test
mean_squared_error(ytest, ypred3)

53852949048.473946

In [99]:
#Lo mismo para train
ytrainpred3 = regresor_svr_lin.predict(xtrain_scal)

In [100]:
# calculo el RMSE (root mean square error) del train
np.sqrt(mean_squared_error(ytrain, ytrainpred3))

247435.93441186982

In [101]:
# calculo el MAE (mean absolute error) del train
mean_absolute_error(ytrain, ytrainpred3)

88456.14038116703

In [102]:
# Gaussiano

In [103]:
regresor_svr_rbf.fit(xtrain_scal,ytrain.ravel())

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  7.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [75, 150], 'gamma': [0.005, 0.01],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [104]:
regresor_svr_rbf.best_params_

{'C': 150, 'gamma': 0.01, 'kernel': 'rbf'}

In [105]:
regresor_svr_rbf.best_score_

-0.024298530912573967

In [106]:
ypred4 = regresor_svr_rbf.predict(xtest_scal)

In [107]:
# calculo el RMSE (root mean square error) del test
np.sqrt(mean_squared_error(ytest, ypred4))

287859.34696704487

In [108]:
# calculo el MAE (mean absolute error) del test
mean_absolute_error(ytest, ypred4)

139222.24277039722

In [109]:
# calculo el MSE (mean square error) del test
mean_squared_error(ytest, ypred4)

82863003636.29352

In [110]:
#Lo mismo para train
ytrainpred4 = regresor_svr_rbf.predict(xtrain_scal)

In [111]:
# calculo el RMSE (root mean square error) del train
np.sqrt(mean_squared_error(ytrain, ytrainpred4))

300998.3721704818

In [112]:
# calculo el MAE (mean absolute error) del train
mean_absolute_error(ytrain, ytrainpred4)

137954.45125974584